In [ ]:
#pip install yfinance --upgrade --no-cache-dir

In [1]:
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

sns.set(style='whitegrid')

In [3]:
acoes = ['ITUB4.SA', 'BBDC4.SA', 'DIRR3.SA', 'JBSS3.SA', 'GUAR3.SA', 'PSSA3.SA','^BVSP']

ydata = yf.download(acoes, '2015-01-01', '2019-12-31', '1d');
dados = ydata["Adj Close"]
dados = dados.rename(columns = {'^BVSP':'IBOV'})
display(dados)

[*********************100%%**********************]  7 of 7 completed


Ticker,BBDC4.SA,DIRR3.SA,GUAR3.SA,ITUB4.SA,JBSS3.SA,PSSA3.SA,IBOV
Date,,,,,,,
2015-01-02,8.713271,4.704646,8.123262,11.957977,7.803239,9.775706,48512.0
2015-01-05,8.731241,4.477846,7.866182,12.018076,7.840223,9.756783,47517.0
2015-01-06,9.018470,4.349908,7.840376,12.212481,7.655310,9.633799,48001.0
2015-01-07,9.376868,4.431324,7.984299,12.654326,7.869807,9.649567,49463.0
2015-01-08,9.425159,4.512738,7.867175,12.852265,7.936375,9.586494,49943.0
...,...,...,...,...,...,...,...
2019-12-20,22.055098,11.318623,20.296730,30.231522,20.531200,25.454185,115121.0
2019-12-23,22.191057,11.526155,20.684278,30.513123,20.783892,25.616083,115863.0
2019-12-26,22.469131,12.068939,21.459377,30.976946,20.918133,25.717270,117203.0


In [5]:
dados = dados *100 / dados.iloc[0]
dados_chg = dados.pct_change()
dados_chg.fillna(0, inplace = True)
dados_chg

C:\Users\Vini\AppData\Local\Temp\ipykernel_5196\2693165503.py:2: FutureWarning: The default fill_method='pad' in DataFrame.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.
  dados_chg = dados.pct_change()


Ticker,BBDC4.SA,DIRR3.SA,GUAR3.SA,ITUB4.SA,JBSS3.SA,PSSA3.SA,IBOV
Date,,,,,,,
2015-01-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2015-01-05,0.002062,-0.048208,-0.031647,0.005026,0.004740,-0.001936,-0.020510
2015-01-06,0.032897,-0.028571,-0.003281,0.016176,-0.023585,-0.012605,0.010186
2015-01-07,0.039740,0.018717,0.018357,0.036180,0.028019,0.001637,0.030458
2015-01-08,0.005150,0.018373,-0.014669,0.015642,0.008459,-0.006536,0.009704
...,...,...,...,...,...,...,...
2019-12-20,-0.005747,-0.013222,0.011226,-0.004364,-0.009524,0.009957,-0.000087
2019-12-23,0.006165,0.018336,0.019094,0.009315,0.012308,0.006360,0.006445
2019-12-26,0.012531,0.047091,0.037473,0.015201,0.006459,0.003950,0.011565


In [6]:
#Criação do dataframe com o índice de mercado o ativo livre de risco

indices_chg = dados_chg[['IBOV']]

tlr_anual = 0.12
tlr_diaria = (1+tlr_anual)**(1/252)-1
indices_chg.insert(1, 'SELIC', tlr_diaria)
indices_chg

Ticker,IBOV,SELIC
Date,,
2015-01-02,0.000000,0.00045
2015-01-05,-0.020510,0.00045
2015-01-06,0.010186,0.00045
2015-01-07,0.030458,0.00045
2015-01-08,0.009704,0.00045
...,...,...
2019-12-20,-0.000087,0.00045
2019-12-23,0.006445,0.00045
2019-12-26,0.011565,0.00045


In [7]:
#Remover o IBOV do dataframe dos dados dos ativos

dados.drop(['IBOV'], axis = 'columns', inplace = True)
dados_chg.drop(['IBOV'], axis = 'columns', inplace = True)

In [8]:
def calc_capm(ativo_chg, alr_chg, merc_chg): #ativo escolhido, ativo livre de risco, mercado

  ativo_m_alr = ativo_chg - alr_chg
  merc_m_alr  = merc_chg  - alr_chg

  ativo_np = ativo_m_alr.to_numpy()
  merc_np =  merc_m_alr.to_numpy()

  merc_np2 = sm.add_constant(merc_np)
  model = sm.OLS(ativo_np, merc_np2)
  res = model.fit()

  return res

In [12]:
port_pesos = [0.1666, 0.1666, 0.1666, 0.1666, 0.1666, 0.1666]
port = dados.dot(port_pesos).pct_change()
port.fillna(0, inplace = True)
res = calc_capm(port, indices_chg['SELIC'], indices_chg['IBOV']);
print("Alfa:", round(res.params[0]*252,3) , "Beta:", round(res.params[1], 3))
print("R2:", round(res.rsquared, 2)) #R2 é o coeficiente de determinação. Medida estatística de quão próximos os dados estão da linha de regressão ajustada

Alfa: 0.03 Beta: 0.877
R2: 0.69


**Carteira com máxima utilidade usando dados históricos**

$Z = Cov^{-1.}$($μ - R_f$)

$w_i$ = $Z_i$ /$∑_j$$Z_j$

In [13]:
#Subtrair  a rentabilidade do ativo livre de risco dos retornos históricos

means = dados_chg.mean().to_numpy()
rf = indices_chg['SELIC'].mean()
means_rf = means - rf
means_rf

array([0.00052618, 0.00062159, 0.00058053, 0.00048484, 0.00083566,
       0.00051671])

In [10]:
#Calcular a covariância dos retornos históricos e transformar em uma matriz do numpy

dados_chg.cov()
cov_mat = dados_chg.cov().to_numpy()

In [18]:
# Cálculo do Z
pesos_hist_z = np.matmul(np.linalg.inv(cov_mat), means_rf)

# Normalização para obtenção dos pesos ótimos
pesos_hist_final = pesos_hist_z/np.sum(pesos_hist_z)
pesos_hist_final

NameError: name 'cov_mat' is not defined

In [12]:
port = dados.dot(pesos_hist_final).pct_change()
port.fillna(0, inplace = True)
port
res = calc_capm(port, indices_chg['SELIC'], indices_chg['IBOV']);
print("Alfa:", round(res.params[0]*252,3) , "Beta:", round(res.params[1], 3))
print("R2:", round(res.rsquared, 2)) #R2 é o coeficiente de determinação. Medida estatística de quão próximos os dados estão da linha de regressão ajustada

Alfa: 0.046 Beta: 0.713
R2: 0.46


In [13]:
dados['PORT1'] = dados.dot(pesos_hist_final)
dados_chg = dados.pct_change().fillna(0)
ret_aa = ((dados.iloc[-1]/dados.iloc[0])**(1/5))-1
print("Ret aa:\n", ret_aa)
vol_aa = dados_chg.std()*np.sqrt(252)
print("Vol aa:\n", vol_aa)
dados=dados.drop(['PORT1'], axis=1)

Ret aa:
 Ticker
BBDC4.SA    0.207330
DIRR3.SA    0.205414
GUAR3.SA    0.218396
ITUB4.SA    0.207749
JBSS3.SA    0.211594
PSSA3.SA    0.210361
PORT1       0.211661
dtype: float64
Vol aa:
 Ticker
BBDC4.SA    0.331645
DIRR3.SA    0.400726
GUAR3.SA    0.344750
ITUB4.SA    0.297338
JBSS3.SA    0.507458
PSSA3.SA    0.316442
PORT1       0.230357
dtype: float64


**Carteira com máxima utilidade usando CAPM**

In [14]:
#Calcular os betas dos ativos

betas = []
for item in range(6):
  vect = [0]*6
  vect[item]=1
  port = dados.dot(vect).pct_change()
  port.fillna(0, inplace = True)
  res = calc_capm(port, indices_chg['SELIC'], indices_chg['IBOV'])
  print("Alfa:", round(res.params[0]*252,3) , "Beta:", round(res.params[1], 3))
  betas.append(round(res.params[1], 3))
betas

Alfa: 0.021 Beta: 1.273
Alfa: 0.085 Beta: 0.817
Alfa: 0.091 Beta: 0.636
Alfa: 0.021 Beta: 1.153
Alfa: 0.145 Beta: 0.752
Alfa: 0.081 Beta: 0.559


[1.273, 0.817, 0.636, 1.153, 0.752, 0.559]

In [15]:
# Calcular o retorno estimado dos ativos usando o CAPM
ret_capm = rf + (np.array(betas) * ((indices_chg['IBOV'].mean()) - rf))

 # Cálculo do Z
pesos_capm_z = np.matmul(np.linalg.inv(cov_mat), ret_capm)

# Normalização para obtenção dos pesos ótimos
pesos_capm_final = pesos_capm_z/np.sum(pesos_capm_z)
pesos_capm_final

array([0.06289222, 0.0940443 , 0.17547309, 0.35703036, 0.0968866 ,
       0.21367343])

In [16]:
port = dados.dot(pesos_capm_final).pct_change()
port.fillna(0, inplace = True)
port
res = calc_capm(port, indices_chg['SELIC'], indices_chg['IBOV']);
print("Alfa:", round(res.params[0]*252,3) , "Beta:", round(res.params[1], 3))
print("R2:", round(res.rsquared, 2)) #R2 é o coeficiente de determinação. Medida estatística de quão próximos os dados estão da linha de regressão ajustada

Alfa: 0.029 Beta: 0.89
R2: 0.72


In [17]:
dados['PORT1'] = dados.dot(pesos_capm_final)
dados_chg = dados.pct_change().fillna(0)
ret_aa = ((dados.iloc[-1]/dados.iloc[0])**(1/5))-1
print("Ret aa:\n", ret_aa)
vol_aa = dados_chg.std()*np.sqrt(252)
print("Vol aa:\n", vol_aa)
dados=dados.drop(['PORT1'], axis=1)

Ret aa:
 Ticker
BBDC4.SA    0.207330
DIRR3.SA    0.205414
GUAR3.SA    0.218396
ITUB4.SA    0.207749
JBSS3.SA    0.211594
PSSA3.SA    0.210361
PORT1       0.210330
dtype: float64
Vol aa:
 Ticker
BBDC4.SA    0.331645
DIRR3.SA    0.400726
GUAR3.SA    0.344750
ITUB4.SA    0.297338
JBSS3.SA    0.507458
PSSA3.SA    0.316442
PORT1       0.229972
dtype: float64
